# Experimenting with OadTR

this notebook is dedicated to conduct training and evaluation of the OadTR model using the METEOR dataset.

## imports and device specification

In [1]:
import json
import time
import numpy as np
import util as utl
import os
import utils

import transformer_models
from custom_dataset import METEORDataLayer
from train import train_one_epoch, evaluate
from test import test_one_epoch
import torch.nn as nn
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'selected device is {device}')

ModuleNotFoundError: No module named 'util'

## training

### model set up

In [ ]:
args = {
    'num_layers': 3,
    'decoder_layers': 5,
    'enc_layers': 64,
    'output_dir': 'models/en_3_decoder_5_lr_drop_1'
}

with open('pvc-meteor/features/data_info_new.json', 'r') as f:
    data_info = json.load(f)['THUMOS']
args.train_session_set = data_info['train_session_set']
args.test_session_set = data_info['test_session_set']
args.class_index = data_info['class_index']
args.numclass = len(args.class_index)
if args.output_dir:
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)


model = transformer_models.VisionTransformer_v3(
    args=args, 
    img_dim=1024, # may need to change to 2 * 1024
    patch_dim=32, # 1027/64 = 16 patches will be generated
    out_dim=7, # nr of classes
    embedding_dim= 1024 # 
    num_heads=4 # optimal according to OadTR paper
    num_layers=3, # optimal according to OadTR paper
    hidden_dim=4096,
    dropout_rate=args.dropout_rate,
    attn_dropout_rate=args.attn_dropout_rate,
    num_channels= 2,
    positional_encoding_type=args.positional_encoding_type
    )

model.to(device)

## evaluation

### load model